In [1]:
import os
import re
import json
import jsonlines
from langchain.schema import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_upstage import UpstageEmbeddings
from langchain_milvus.vectorstores import Milvus
from uuid import uuid4
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

import pandas as pd
import pytz

from datasets import Dataset
from ragas import evaluate
import datetime
from datetime import timedelta
from operator import itemgetter

c:\Users\Jo\AppData\Local\Programs\Python311\Lib\site-packages\ragas\metrics\__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
c:\Users\Jo\AppData\Local\Programs\Python311\Lib\site-packages\ragas\metrics\__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no 

In [21]:
def get_pdf_paths(directory):
    pdf_paths = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jsonl'):
                full_path = os.path.join(root, file)
                pdf_paths.append(full_path)

    return pdf_paths

# 사용 예시
directory_path = "./parsed_pdf/result_v11"
result = get_pdf_paths(directory_path)

In [22]:
result

['./parsed_pdf/result_v11\\text\\KISWeekly제1090호(20240628)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1091호(20240705)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1092호(20240712)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1093호(20240719)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1094호(20240726)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1095호(20240802)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1096호(20240809)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1097호(20240816)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1098호(20240823)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1099호(20240830)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1100호(20240906)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1101호(20240913)_docs_text.jsonl',
 './parsed_pdf/result_v11\\text\\KISWeekly제1103호(20240927)_docs_text.jsonl',

In [5]:
document = []
for filepath in result:
    documents = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            # 각 줄을 JSON으로 파싱
            if line.startswith('\n('):
                continue
            data = json.loads(line)

            doc = Document(
                page_content=data['page_content'],
                metadata=data['metadata']
            )
            documents.append(doc)
    document.append(documents)

In [6]:
document = [j for i in document for j in i]

In [7]:
document

[Document(metadata={'type': 'text', 'page': 8, 'source': './raw_pdf_copy3/KISWeekly제1090호(20240628).pdf', 'summary': '- 국공채 시장은 약세로 시작, 외국인 매수세로 일부 회복.\n- 호주 5월 CPI가 전년 대비 4.0% 상승, 예상치 초과로 호주 국채 약세.\n- 국고채 20년 금리는 3.220%에 7천억 원 낙찰, 2조 1720억 원 응찰.\n- 28일 국고채 3년 최종호가는 3.182%, 10년물은 3.266%로 각각 3.8bp, 3.4bp 하락.\n- 국고 3년물은 지난주 대비 1.7bp 상승, 5년물은 0.5bp 상승, 10년물은 1.8bp 상승.\n- 국고 3/5년 스프레드는 1.8bp로 1.2bp 축소, 5/10년 스프레드는 6.5bp, 3/10년 스프레드는 8.3bp로 0.1bp 확대.', 'issue_date': '20240628'}, page_content='주 첫 영업일 국내 채권시장은 지난주 급격하게 하락한 금리를 되돌리며 약세로 출발했다. 장중 외 국인 매수세 지속으로 약세가 일부 되돌려졌으나, 전반적인 아시아 채권시장의 약세 분위기로 국내 채 권시장 또한 약세 우위를 보였다. 익영업일 미국 채권시장 강세의 영향으로 국고채 금리 또한 하락 출 발했다. 이후 특별한 재료 없이 수급에 의존하는 모습을 보이며 장기물을 중심으로 소폭 강세를 보였 다. 이날 국고 20년은 3.220%에 7천억 원이 낙찰, 2조 1720억 원이 응찰되었다. 26일 호주 5월 CPI가 전년대비 4.0% 상승하는 등 예상치를 상회하며 호주 국채가 큰 폭의 약세를 보였다. 이에 연동되어 국 채선물 또한 하락세를 보였고, 지난주 강세가 지속적으로 조정되는 모습을 보였다. 주 마지막 날 채권 시장은 미 국채와 연동되어 강세로 출발했지만 미 대선 토론의 영향으로 낙폭을 축소했고, 미국 5월 PCE 발표를 앞두고 경계감을 보이며 강세로 마감했다. 28일 국고채 3년 최종호가 수익률은

In [2]:
embeddings = UpstageEmbeddings(
    model='solar-embedding-1-large-query',
)

In [3]:
def save_docs_to_jsonl(documents, file_path):
    with jsonlines.open(file_path, mode="w") as writer:
        for doc in documents:
            writer.write(doc.dict())

In [4]:
filepath = './chunked_jsonl/text_semantic_per_80.jsonl'

splitted_doc = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        splitted_doc.append(doc)

In [5]:
URI = 'http://127.0.0.1:19530/'

vectorstore_text = Milvus(
    embedding_function=embeddings,
    connection_args={'uri':URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_semantic_per_80'
)

2025-03-12 16:11:11,766 [ERROR][_create_connection]: Failed to create new connection using: 1aea6a5381194baf859599912beda974 (milvus_client.py:920)


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19530, illegal connection params or server unavailable)>

In [ ]:
# # 최초 1회 실행 후 주석 처리
# uuids = [str(uuid4()) for _ in range(len(splitted_doc))]

# vectorstore_text.add_documents(
#     documents=splitted_doc,
#     ids = uuids
# )

['2aaa997f-686c-49f9-baba-527a53b45eb1',
 '023de705-2baa-489e-980f-734efc07993f',
 'df1112a3-b6bc-4ac4-b6dd-16c0a6997484',
 'a69cb961-f6f9-40ea-bab2-143e37f63ee4',
 '4702f6ce-890d-480f-93ef-486d69656347',
 '8af744a0-048b-4ac7-9d23-83c6e6e2c24b',
 'a4a70df6-b5c8-4543-9491-f087b3e2ec5a',
 'e8963653-5d06-4b4d-9856-0bbbb84af285',
 '1710ac33-0ebf-49f5-b171-39c3564947c0',
 '5d07bee3-5625-4972-948e-757755a3f0de',
 '88bc38fe-fbc4-4234-8984-433b79915c75',
 '206a73e9-81f7-4b1a-990a-444edcbd598f',
 '62cc2497-3b7d-4a15-ad76-8569fe93a5c2',
 '3a87d788-3346-4ba9-a9f9-f08af6f95fcf',
 '5359c070-5bfc-4038-90ce-7376cc1887c2',
 '09cd794b-cadf-4cbc-8fde-207ab08b7aad',
 '07da2586-f6a8-47db-8fe6-8b8f79f0f1a8',
 '84fc5458-6d8b-48ff-be0b-bf0bf0c8d38e',
 'c1f9df7f-e464-4545-a671-c489109af8cd',
 '6978ff5f-61d4-4c6f-8365-aa6fdef93195',
 '4977534d-8f8f-4363-82ff-ab1daedd7e98',
 'bb61ece0-e151-4a29-9697-120a1fd57274',
 '2495ef5e-12e2-4462-a107-27333df76d67',
 '5fe9a0c9-203b-40e4-9fd7-ee866a7a0cfd',
 'db6a35cd-382a-

In [ ]:
def adjust_time_filter_to_week(time_filter):
    """
    특정 날짜(YYYY-MM-DD)가 주어진 경우, 해당 날짜를 포함하는 주(월~일)의
    첫 번째 날(월요일)과 마지막 날(일요일)로 변환하는 함수.

    :param time_filter: dict, {"start_date": datetime, "end_date": datetime}
    :return: dict, {"start_date": datetime, "end_date": datetime}
    """
    # Extract start_date and end_date from time_filter
    start_date = time_filter.start_date
    end_date = time_filter.end_date

    # Handle the case where start_date or end_date is None
    if start_date is None or end_date is None:
        if start_date is not None and end_date is None:
            start_of_week = start_date - timedelta(days=start_date.weekday())  # 월요일 찾기
            end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

            return {
                "start_date": start_of_week.replace(hour=0, minute=0, second=0),
                "end_date": end_of_week.replace(hour=23, minute=59, second=59)
            }
        elif end_date is not None and start_date is None:
            start_of_week = end_date - timedelta(days=end_date.weekday())  # 월요일 찾기
            end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

            return {
                "start_date": start_of_week.replace(hour=0, minute=0, second=0),
                "end_date": end_of_week.replace(hour=23, minute=59, second=59)
            }
        else:
            return None  # or return the time_filter as is if you prefer

    # 날짜가 동일한 경우, 주의 첫 번째 날(월요일)과 마지막 날(일요일)로 변경
    if start_date.year == end_date.year and start_date.month==end_date.month and start_date.day==end_date.day:
        start_of_week = start_date - timedelta(days=start_date.weekday())  # 월요일 찾기
        end_of_week = start_of_week + timedelta(days=6)  # 해당 주 일요일 찾기

        return {
            "start_date": start_of_week.replace(hour=0, minute=0, second=0),
            "end_date": end_of_week.replace(hour=23, minute=59, second=59)
        }

    # 날짜가 다르면 기존 time_filter 유지
    return {
        "start_date": start_date,
        "end_date": end_date
    }

In [8]:
df_qa_date = pd.read_csv('./QA/qa_ragas_date.csv')

# 문자열을 datetime 객체로 변환하는 함수
def parse_datetime_str(data):
    result = []
    for item in data:
        if pd.isna(item):
            result.append(None)
        else:
            try:
                date_pattern = r"datetime\.datetime\((.*?)\)"
                tzinfo_pattern = r"TzInfo\((.*?)\)"

                # datetime 객체를 변환하는 함수
                def parse_datetime(match):
                    if match:
                        # 'datetime.datetime(2024, 11, 1, 0, 0)' 형태로 되어있으므로, 이를 eval로 처리
                        date_str = match.group(1)
                        # 날짜를 문자열로 변환
                        return eval(f"datetime.datetime({date_str})")
                    return match.group(0)

                # tzinfo 처리
                def parse_tzinfo(match):
                    if match:
                        # TzInfo(UTC) 형태로 되어있으므로, 이를 pytz.UTC로 변환
                        tz_str = match.group(1)
                        return pytz.UTC if tz_str == 'UTC' else None
                    return match.group(0)

                # 'datetime' 및 'tzinfo'를 처리
                item = re.sub(date_pattern, parse_datetime, item)
                item = re.sub(tzinfo_pattern, parse_tzinfo, item)

                # 최종적으로 변환된 문자열을 eval로 실행하여 객체로 만듦
                item = eval(item)

                result.append(item)
            except Exception as e:
                result.append(None)
                print(f"Error parsing item: {item}, Error: {e}")
    return result

# 'date' 열에서 각 항목을 변환하여 새로운 리스트로 생성
parsed_dates = parse_datetime_str(df_qa_date['date'])

In [9]:
# 결과를 저장할 리스트
start_dates = []
end_dates = []

# parsed_dates를 순회하며 None인 경우도 처리
for date_entry in parsed_dates:
    if date_entry:
        start_dates.append(date_entry['start_date'])
        end_dates.append(date_entry['end_date'])
    else:
        start_dates.append(None)
        end_dates.append(None)

In [10]:
def convert_to_list(example):
    if isinstance(example["contexts"], list):  # 이미 리스트이면 그대로 반환
        contexts = example["contexts"]
    else:
        try:
            contexts = json.loads(example["contexts"])  # JSON 문자열이면 변환
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error: {example['contexts']} - {e}")
            contexts = []  # 예외 발생 시 빈 리스트로 대체
    return {"contexts": contexts}

In [11]:
expressions = []

for start, end in zip(start_dates, end_dates):
    if start is None or end is None:
        expressions.append(None)
    else:
        expr = f"issue_date > '{start.strftime('%Y%m%d')}' AND issue_date < '{end.strftime('%Y%m%d')}'"
        expressions.append(expr)

for expr in expressions:
    print(expr)

issue_date > '20241101' AND issue_date < '20241130'
None
issue_date > '20240901' AND issue_date < '20240930'
None
issue_date > '20241216' AND issue_date < '20241222'
issue_date > '20241101' AND issue_date < '20241130'
issue_date > '20241101' AND issue_date < '20241130'
issue_date > '20240901' AND issue_date < '20240907'
issue_date > '20240624' AND issue_date < '20240630'
issue_date > '20241001' AND issue_date < '20241031'
None
issue_date > '20240729' AND issue_date < '20240804'
issue_date > '20241125' AND issue_date < '20241201'
issue_date > '20240722' AND issue_date < '20240728'
issue_date > '20241001' AND issue_date < '20241031'
issue_date > '20241209' AND issue_date < '20241215'
issue_date > '20250113' AND issue_date < '20250119'
issue_date > '20241201' AND issue_date < '20241207'
issue_date > '20241011' AND issue_date < '20241018'
issue_date > '20220101' AND issue_date < '20221231'
issue_date > '20240909' AND issue_date < '20240915'
issue_date > '20241222' AND issue_date < '2024122

In [12]:
from langchain_teddynote.retrievers import KiwiBM25Retriever

In [14]:
embeddings=UpstageEmbeddings(
    model='solar-embedding-1-large-query'
)

vectorstore_text = Milvus(
    embedding_function=embeddings,
    connection_args={'uri':URI},
    index_params={'index_type': 'AUTOINDEX', 'metric_type': 'IP'},
    collection_name='text_semantic_per_80'
)

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

milvus_retriever = vectorstore_text.as_retriever(
    search_kwargs={'k':10}
)

bm25 = KiwiBM25Retriever.from_documents(
  splitted_doc
)
bm25.k = 50

bm25_2 = KiwiBM25Retriever.from_documents(
    splitted_doc
)
bm25_2.k = 5

df_qa = pd.read_csv("./QA/qa_ragas.csv", index_col=False)
df_qa = df_qa[['question', 'ground_truth']]
df_qa.ground_truth = df_qa.apply(lambda x: x.ground_truth.split("'")[1].split("'")[0], axis=1)

# df_qa_date = pd.read_csv('./QA/qa_ragas_date.csv', index_col=False)
test_dataset = Dataset.from_pandas(df_qa)

In [ ]:
prompt = PromptTemplate.from_template(
'''You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer simply.
Answer in Korean.

#Question:
{question}
#Context:
{context}

#Answer:'''
)

In [35]:
from pymilvus.model.reranker import CrossEncoderRerankFunction

# Define the rerank function
ce_rf = CrossEncoderRerankFunction(
    model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",  # Specify the model name.
    device="cpu" # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)

c:\Users\Jo\AppData\Local\Programs\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [89]:
result_ = []
batch_dataset = [question for question in test_dataset["question"]]
context = []
answer = []
invoked_list = []

for expr, question in zip(expressions, batch_dataset):
    invoked_milvus = milvus_retriever.invoke(question, expr=expr)
    invoked_bm25 = bm25.invoke(question)
    invoked_filtered = []
    for i in invoked_bm25:
        if expr != None:
            if i.metadata['issue_date'] >= expr.split("'")[1] and i.metadata['issue_date'] <= expr.split("'")[3]:
                invoked_filtered.append(i)
        else:
            invoked_filtered.append(i)
    invoked_bm25_2 = bm25_2.invoke(question)
    invoked =  invoked_milvus + invoked_bm25_2 + invoked_filtered
    invoked_list.append(invoked)
    document_texts = [doc.page_content for doc in invoked]
    reranked_results = ce_rf(
        query=question,
        documents=document_texts,
        top_k=5
    )
    context.append(reranked_results)
    chain = (
        {'context' : itemgetter('context'), 'question': itemgetter('question')}
        # | add_date_to_inputs
        | prompt
        | llm
        | StrOutputParser()
    )
    answer.append(chain.invoke({'question':question, 'context': invoked}))

In [ ]:
if "contexts" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["contexts"]).add_column("contexts", [[j.text for j in i] for i in context])
else:
    test_dataset = test_dataset.add_column("contexts", [[j.text for j in i] for i in context])

if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

test_dataset = test_dataset.map(convert_to_list)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [18]:
from ragas import evaluate
from ragas.metrics import (
    context_recall,
    context_precision,
    AnswerCorrectness,
    answer_similarity
)

for _ in range(3):
    result = evaluate(
        dataset=test_dataset,
        metrics=[
            context_recall,
            context_precision,
            answer_similarity,
            AnswerCorrectness(weights=[0.75, 0.25]),
        ],
    )
    result_.append(result)

with open('./score/250310_11.json', "w", encoding='utf-8') as json_file:
    json.dump(result_, json_file, indent=4)

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

In [37]:
context

[[RerankResult(text='2024년 11월 조기상환 건수는 지난달 대비 39.31% 감소한 420건을 기록했다. 만기상환 건수는 440건 으로 전월 대비 33.03% 감소했다. 감소, 감소하는 모습을 보였고, 평균지급률의 경우에는 조기상환 평균 지급률은 증가, 만기상환 평균지급률이 감소하였다. 지난 10월과 비교하여 수익 상환건수는 약 675건으로 42.46% 감소하였고, 원금 상환 건수는 173건을 기록하였으며, 손실상환건수는 12건을 기록하며 지난달 대비 감소하는 모습을 보였다.', score=7.525996208190918, index=23),
  RerankResult(text='이번주 ABS는 단말기할부대금채권 기초 티월드제일백오차가 총 3,500억원 발행되었다. 또한 Primary CBO 기초 신보 24~27회가 총 4,800억원 가량 발행되었다.', score=7.447983741760254, index=4),
  RerankResult(text='2024년 11월에는 전월 대비 발행 건수는 감소하였고, 발행총액은 증가하는 경향을 보였다. 총 866건의 ELS가 발행되었고 발행총액은 약 39,044억원이다. 발행건수는 10월 대비 19.52% 감소하였고, 총 발행액 은 20.25% 증가하였다. 2024년 11월 공모 ELS는 798건으로 전월 대비 19.15% 감소했고 발행총액은 약 37,347억원을 기록하며 지난달 대비 20.66% 증가하였다. 이 중 연금상품의 발행건수는 203건으로 전월(212건) 대비 4.25% 감소 하였고, 발행액은 23,173억원으로 전월 대비 79.05% 증가하였다. 또한 사모 ELS는 68건으로 전월 대비 23.6% 감소하였고, 발행총액은 약 1,697억원을 기록하며 11.79% 증가하였다.', score=7.404268264770508, index=20),
  RerankResult(text='2024년 11월 ELS 상환평균지급률은 지난달(6.14%)보다 0.39%포인트 감소한 5.76%이며, ELS 

In [38]:
context[-1]

[RerankResult(text='이번주 ABS 발행은 전무하였다. 한편 MBS는 7,351원 대상으로 7,504억원(선순위 7,476 억원 및 후순위 28 억원) 발행하였다.', score=8.70724868774414, index=14),
 RerankResult(text='이번주 ABS 및 MBS 발행은 전무하였다. 부동산 경기 침체로 인한 PF 부문의 부진 예상과 기준금리 인하 기 대감에 따른 정기예금 수요감소로 PF대출채권 및 정기예금 기초자산 유동화증권이 축소되는 모습이다.', score=8.169910430908203, index=1),
 RerankResult(text='이번주 ABS 및 MBS 발행은 전무하였다. 부동산 경기 침체로 인한 PF 부문의 부진 예상과 기준금리 인하 기 대감에 따른 정기예금 수요감소로 PF대출채권 및 정기예금 기초자산 유동화증권이 축소되는 모습이다.', score=8.169910430908203, index=13),
 RerankResult(text='이번주 ABS 및 MBS 발행은 전무하였다. 부동산 경기 침체로 인한 PF 부문의 부진 예상과 기준금리 인하 기 대감에 따른 정기예금 수요감소로 PF대출채권 및 정기예금 기초자산 유동화증권이 축소되는 모습이다.', score=8.169910430908203, index=15),
 RerankResult(text='이번주 ABS 발행시장은 소강상태였다. 한편 MBS 발행시장에서는 6,161억 원을 대상자산으로 하여 6,324 억 원(선순위 6,292억 원 및 후순위 32억 원)의 채권을 발행했다.', score=8.096586227416992, index=12)]

In [39]:
invoked

[Document(metadata={'source': './raw_pdf_copy3/KISWeekly제1091호(20240705).pdf', 'summary': '- 2023-07-01부터의 유동화 발행금액: 70,411억원\n- 전주 대비 감소: 31,737억원\n- 기초자산: 부동산 관련 대출이 가장 많이 발행\n- 정기예금이 그 뒤를 이음\n- 자료 출처: KIS자산평가', 'issue_date': '20240705', 'pk': '9ce4d615-6c30-4118-b0e6-8c40e2647556', 'type': 'text', 'page': 26}, page_content='금주 유동화 발행금액은 70,411억원으로 전주 대비 31,737억원 감소하였다. 기초자산은 부동산 관련 대 출이 가장 많이 발행되었고, 정기예금이 그 뒤를 이었다.'),
 Document(metadata={'source': './raw_pdf_copy3/KISWeekly제1091호(20240705).pdf', 'summary': '- 이번주 ABS 및 MBS 발행 전무\n- 부동산 경기 침체로 PF 부문 부진 예상\n- 기준금리 인하 기대감에 따른 정기예금 수요 감소\n- PF대출채권 및 정기예금 기초자산 유동화증권 축소\n- 자료 출처: KIS자산평가, DART전자공시시스템', 'issue_date': '20240705', 'pk': 'a0a8ee90-827a-4c89-b363-2cbb9db36ce0', 'type': 'text', 'page': 21}, page_content='이번주 ABS 및 MBS 발행은 전무하였다. 부동산 경기 침체로 인한 PF 부문의 부진 예상과 기준금리 인하 기 대감에 따른 정기예금 수요감소로 PF대출채권 및 정기예금 기초자산 유동화증권이 축소되는 모습이다.'),
 Document(metadata={'source': './raw_pdf_copy3/KISWeekly제1091호(20240705).pdf', 'summary': '- 금주 CD 발행: 12건

In [ ]:
answ = []
for i in context[-1]:
  for j in invoked:
    if i.text == j.page_content:
      answ.append(j)

In [41]:
a = set()
for i in answ:
  a.add((i.metadata['page'], i.metadata['source']))

In [42]:
for i, j in a:
  print(i, j)

22 ./raw_pdf_copy3/KISWeekly제1092호(20240712).pdf
21 ./raw_pdf_copy3/KISWeekly제1091호(20240705).pdf
22 ./raw_pdf_copy3/KISWeekly제1097호(20240816).pdf


In [43]:
filepath = './chunked_jsonl/image_v2.jsonl'

splitted_doc = []
with open(filepath, 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('\n('):
            continue
        data = json.loads(line)

        doc = Document(
            page_content=data['page_content'],
            metadata=data['metadata']
        )
        splitted_doc.append(doc)

In [45]:
b = []
for i, j in a:
    for k in splitted_doc:
        if k.metadata['page'] == i and k.metadata['source'] == j:
            b.append(k)

In [46]:
b

[Document(metadata={'type': 'image', 'image': './raw_pdf_copy3/KISWeekly제1092호(20240712)\\290.png', 'page': 22, 'source': './raw_pdf_copy3/KISWeekly제1092호(20240712).pdf', 'id': 290, 'issue_date': '20240712'}, page_content='2024년 1월의 ABS 발행금액은 약 2,000으로 가장 낮고, 2024년 2월에는 약 14,000으로 급증하며 최고치를 기록했으며, 이후 2024년 3월과 4월에도 비슷한 수준을 유지하고, 2024년 5월에는 약 10,000으로 감소한 후, 2024년 6월에는 다시 약 12,000으로 증가하는 추세를 보인다.'),
 Document(metadata={'type': 'image', 'image': './raw_pdf_copy3/KISWeekly제1092호(20240712)\\291.png', 'page': 22, 'source': './raw_pdf_copy3/KISWeekly제1092호(20240712).pdf', 'id': 291, 'issue_date': '20240712'}, page_content='월간 기초자산 별 ABS 발행금액 추이에 따르면, PF Loan이 가장 높은 발행금액을 기록했으며, P-CBO와 기타는 중간 수준의 발행금액을 보였다. 오토론과 단말기할부대출채권은 상대적으로 낮은 발행금액을 나타냈고, 확정/장래매출채권 CDO와 Synthetic CDO는 가장 낮은 발행금액을 기록했다. 전체적으로 발행잔액은 지속적으로 증가하는 추세를 보였다.'),
 Document(metadata={'type': 'image', 'image': './raw_pdf_copy3/KISWeekly제1092호(20240712)\\295.png', 'page': 22, 'source': './raw_pdf_copy3/KISWeekly제1092호(20240712).pdf',

In [56]:
from langchain_teddynote.evaluator import GroundednessChecker

question_answer_relevant = GroundednessChecker(
  llm=ChatOpenAI(model='gpt-4o-mini', temperature=0), target='retrieval-answer'
).create()

response_ = []
for j in answer:
    response = []
    for i in b:
        response.append(question_answer_relevant.invoke(
        {'context': i, 'answer': answer[-1]}
        ))
    response_.append(response)

KeyboardInterrupt: 

In [ ]:
response

[GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no'),
 GroundnessAnswerRetrievalScore(score='no')]

In [61]:
context

[[RerankResult(text='2024년 11월 조기상환 건수는 지난달 대비 39.31% 감소한 420건을 기록했다. 만기상환 건수는 440건 으로 전월 대비 33.03% 감소했다. 감소, 감소하는 모습을 보였고, 평균지급률의 경우에는 조기상환 평균 지급률은 증가, 만기상환 평균지급률이 감소하였다. 지난 10월과 비교하여 수익 상환건수는 약 675건으로 42.46% 감소하였고, 원금 상환 건수는 173건을 기록하였으며, 손실상환건수는 12건을 기록하며 지난달 대비 감소하는 모습을 보였다.', score=7.525996208190918, index=23),
  RerankResult(text='이번주 ABS는 단말기할부대금채권 기초 티월드제일백오차가 총 3,500억원 발행되었다. 또한 Primary CBO 기초 신보 24~27회가 총 4,800억원 가량 발행되었다.', score=7.447983741760254, index=4),
  RerankResult(text='2024년 11월에는 전월 대비 발행 건수는 감소하였고, 발행총액은 증가하는 경향을 보였다. 총 866건의 ELS가 발행되었고 발행총액은 약 39,044억원이다. 발행건수는 10월 대비 19.52% 감소하였고, 총 발행액 은 20.25% 증가하였다. 2024년 11월 공모 ELS는 798건으로 전월 대비 19.15% 감소했고 발행총액은 약 37,347억원을 기록하며 지난달 대비 20.66% 증가하였다. 이 중 연금상품의 발행건수는 203건으로 전월(212건) 대비 4.25% 감소 하였고, 발행액은 23,173억원으로 전월 대비 79.05% 증가하였다. 또한 사모 ELS는 68건으로 전월 대비 23.6% 감소하였고, 발행총액은 약 1,697억원을 기록하며 11.79% 증가하였다.', score=7.404268264770508, index=20),
  RerankResult(text='2024년 11월 ELS 상환평균지급률은 지난달(6.14%)보다 0.39%포인트 감소한 5.76%이며, ELS 

In [71]:
invoked_list

[[Document(metadata={'type': 'text', 'page': 18, 'source': './raw_pdf_copy3\\KISWeekly제1112호(20241129).pdf', 'summary': '- 이번 주 회사채 발행금액: 1조 5,410억 원\n- 만기금액: 6,700억 원\n- 순 발행: 800억 원\n- 발행 등급: 초우량등급부터 비우량등급까지 다양\n- 한화오션(BBB+): 신용 등급 상향 후 첫 공모채 발행에서 목표액의 8배 자금 모집, 언더금리 발행 성공\n- 케이티(AAA): 2,000억 원 모집\n- SK브로드밴드(AA): 1,500억 원 모집\n- 효성화학(BBB+): 세 번째 미매각 기록', 'issue_date': '20241129', 'pk': '5194595e-7f5b-4800-b5f2-2a4ab28ea6ab'}, page_content='특히 한화오션(BBB+)는 신용 등급 상향이후 첫 공모채 발행에서 목표액의 8배가 넘는 자금이 몰리며 언더금리 발행에 성공했다. 수요예측시장에서는 케이티(AAA), SK브로드밴드(AA)가 각각 2,000억 원, 1,500억 원을 모집했고 민평대비 소폭 언더발행된 수준으로 수요예측을 마쳤다. 그러나 효성화학(BBB+)은 이번에도 미매각 되면서 연속 세 번째 미매각을 기록했다.'),
  Document(metadata={'type': 'text', 'page': 21, 'source': './raw_pdf_copy3\\KISWeekly제1110호(20241115).pdf', 'summary': '- 이번주 ABS 발행: Primary CBO를 기초자산으로 한 2024기보제이차유동화증권 발행\n- 발행 규모: 총 5,037억 원 (선순위 5,063억 원, 후순위 12억 원)\n- 기초자산: 165개 기업 발행 무보증회사채\n- 참여 증권사: 코리아에셋투자증권㈜, 아이비케이투자증권㈜, ㈜아이엠증권, 케이비증권㈜, 유안타증권㈜, 키움증권㈜, 현대차증권㈜\n- 발행된 유동화사채: 제1-1회

In [75]:
answ

[[Document(metadata={'type': 'text', 'page': 18, 'source': './raw_pdf_copy3\\KISWeekly제1112호(20241129).pdf', 'summary': '- 이번 주 회사채 발행금액: 1조 5,410억 원\n- 만기금액: 6,700억 원\n- 순 발행: 800억 원\n- 발행 등급: 초우량등급부터 비우량등급까지 다양\n- 한화오션(BBB+): 신용 등급 상향 후 첫 공모채 발행에서 목표액의 8배 자금 모집, 언더금리 발행 성공\n- 케이티(AAA): 2,000억 원 모집\n- SK브로드밴드(AA): 1,500억 원 모집\n- 효성화학(BBB+): 세 번째 미매각 기록', 'issue_date': '20241129', 'pk': '5194595e-7f5b-4800-b5f2-2a4ab28ea6ab'}, page_content='특히 한화오션(BBB+)는 신용 등급 상향이후 첫 공모채 발행에서 목표액의 8배가 넘는 자금이 몰리며 언더금리 발행에 성공했다. 수요예측시장에서는 케이티(AAA), SK브로드밴드(AA)가 각각 2,000억 원, 1,500억 원을 모집했고 민평대비 소폭 언더발행된 수준으로 수요예측을 마쳤다. 그러나 효성화학(BBB+)은 이번에도 미매각 되면서 연속 세 번째 미매각을 기록했다.'),
  Document(metadata={'type': 'text', 'page': 21, 'source': './raw_pdf_copy3\\KISWeekly제1110호(20241115).pdf', 'summary': '- 이번주 ABS 발행: Primary CBO를 기초자산으로 한 2024기보제이차유동화증권 발행\n- 발행 규모: 총 5,037억 원 (선순위 5,063억 원, 후순위 12억 원)\n- 기초자산: 165개 기업 발행 무보증회사채\n- 참여 증권사: 코리아에셋투자증권㈜, 아이비케이투자증권㈜, ㈜아이엠증권, 케이비증권㈜, 유안타증권㈜, 키움증권㈜, 현대차증권㈜\n- 발행된 유동화사채: 제1-1회

[[RerankResult(text='2024년 11월 조기상환 건수는 지난달 대비 39.31% 감소한 420건을 기록했다. 만기상환 건수는 440건 으로 전월 대비 33.03% 감소했다. 감소, 감소하는 모습을 보였고, 평균지급률의 경우에는 조기상환 평균 지급률은 증가, 만기상환 평균지급률이 감소하였다. 지난 10월과 비교하여 수익 상환건수는 약 675건으로 42.46% 감소하였고, 원금 상환 건수는 173건을 기록하였으며, 손실상환건수는 12건을 기록하며 지난달 대비 감소하는 모습을 보였다.', score=7.525996208190918, index=23),
  RerankResult(text='이번주 ABS는 단말기할부대금채권 기초 티월드제일백오차가 총 3,500억원 발행되었다. 또한 Primary CBO 기초 신보 24~27회가 총 4,800억원 가량 발행되었다.', score=7.447983741760254, index=4),
  RerankResult(text='2024년 11월에는 전월 대비 발행 건수는 감소하였고, 발행총액은 증가하는 경향을 보였다. 총 866건의 ELS가 발행되었고 발행총액은 약 39,044억원이다. 발행건수는 10월 대비 19.52% 감소하였고, 총 발행액 은 20.25% 증가하였다. 2024년 11월 공모 ELS는 798건으로 전월 대비 19.15% 감소했고 발행총액은 약 37,347억원을 기록하며 지난달 대비 20.66% 증가하였다. 이 중 연금상품의 발행건수는 203건으로 전월(212건) 대비 4.25% 감소 하였고, 발행액은 23,173억원으로 전월 대비 79.05% 증가하였다. 또한 사모 ELS는 68건으로 전월 대비 23.6% 감소하였고, 발행총액은 약 1,697억원을 기록하며 11.79% 증가하였다.', score=7.404268264770508, index=20),
  RerankResult(text='2024년 11월 ELS 상환평균지급률은 지난달(6.14%)보다 0.39%포인트 감소한 5.76%이며, ELS 

In [102]:
response_ = []
yes_thing = []
for j, k, l in zip(answer, context, invoked_list):
    answ = []
    for i in k:
        for o in l:
            if i.text == o.page_content:
                answ.append(o)
    a = set()
    for i in answ:
        a.add((i.metadata['page'], i.metadata['source']))

    b = []
    for i, m in a:
        for n in splitted_doc:
            if n.metadata['page'] == i and n.metadata['source'] == m:
                b.append(n)

    response = []
    for i in b:
        result =question_answer_relevant.invoke(
        {'context': i, 'answer': j}
        )
        response.append(result)
        if result.score=='yes':
            yes_thing.append(i)
    response_.append(response)

In [103]:
yes_thing

[Document(metadata={'type': 'image', 'image': './raw_pdf_copy3\\KISWeekly제1113호(20241206)\\744.png', 'page': 54, 'source': './raw_pdf_copy3\\KISWeekly제1113호(20241206).pdf', 'id': 744, 'issue_date': '20241206'}, page_content='2023년 3월부터 2024년 11월까지 CD 발행 잔액은 지속적으로 증가하여 2024년 3월에 약 70,000에 도달하고, 이후 안정세를 보이며 60,000대에서 유지되고 있으며, KIS CD 수익률은 2023년 3월에 약 1.5%에서 시작해 2024년 3월에 4%를 초과한 후, 2024년 11월까지 3.5%로 소폭 하락하는 추세를 보인다.'),
 Document(metadata={'type': 'image', 'image': './raw_pdf_copy3\\KISWeekly제1114호(20241213)\\120.png', 'page': 8, 'source': './raw_pdf_copy3\\KISWeekly제1114호(20241213).pdf', 'id': 120, 'issue_date': '20241213'}, page_content='국고채 금리변동 차트에서 2023년 12월부터 2024년 12월까지 KTB 3년, 5년, 10년, 20년 금리가 전반적으로 하락세를 보이며, 특히 KTB 10년 금리는 약 3.5%에서 시작해 3% 이하로 떨어진 반면, KTB 3년 금리는 3%에서 2.5%로 감소하는 경향을 나타냅니다. 기준금리는 3.5%로 유지되며, 전체적으로 금리 간의 차이가 줄어드는 모습을 보입니다.'),
 Document(metadata={'type': 'image', 'image': './raw_pdf_copy3\\KISWeekly제1113호(20241206)\\384.png', 'page': 26, 'source': './raw_pdf_copy3\\KI

In [104]:
#12 15 17 21
response_

[[GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no')],
 [GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no')],
 [GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore(score='no'),
  GroundnessAnswerRetrievalScore

In [108]:
invoked_list[15]

[Document(metadata={'pk': 'a667ad66-9ad4-452d-818b-eae3050465df', 'type': 'text', 'page': 8, 'source': './raw_pdf_copy3\\KISWeekly제1114호(20241213).pdf', 'summary': '- 정치적 불확실성 및 환율 급등 주목\n- 외국인 매수세 지속으로 강세 전환\n- 국고채 금리는 커브 플래트닝으로 마감\n- 국고 3년 및 10년물 금리는 각각 2.541%, 2.676% 기록\n- 국고 3년물은 8.2bp 하락, 5년물은 1.8bp 하락, 10년물은 7.0bp 하락\n- 국고 3/5년 스프레드 7.7bp, 5/10년 스프레드 5.8bp, 3/10년 스프레드 13.5bp 확대', 'issue_date': '20241213'}, page_content='익영업 일 국고채 지표 교체로 인해 국고 3년 및 10년물 금리는 2.5% 중반대를 기록했다. 이후 정치권의 추경 관련 발언으로 국고채 금리는 약세를 보였지만, 외국인 매수세가 재차 주목되며 강세로 마감했다. 주중 대통령 담화 등을 거치며 탄핵 가능성이 높아짐에 따라 국내 채권시장의 변동성 확대가 지속되었다. 주 마지막 영업일 국내 채권시장은 주말 대통령 탄핵 표결 등을 주시하며 경계심을 보였고, 대외금리 연동 에 따른 하락세를 보이며 장기물 중심의 강세로 마감했다. 13일 국고채 3년 최종호가 수익률은 전장 대 비 0.8bp 하락한 2.541%, 10년 최종호가 수익률은 2.4bp 상승한 2.676%로 마감했다.'),
 Document(metadata={'pk': '96703c16-3ccf-4a85-a3d6-e6dce2c0fda4', 'type': 'text', 'page': 8, 'source': './raw_pdf_copy3\\KISWeekly제1114호(20241213).pdf', 'summary': '- 정치적 불확실성 및 환율 급등 주목\n- 외국인 매수세 지속으로 강세 전환\n- 국고채 금리는

In [109]:
context[15]

[RerankResult(text='11일 국내 채권시장은 금통위를 앞두고 약보합세로 시작했다. 금통위 내 기준금리 동결이 결정되었지만 시 장에 영향은 미미했고, 부동산 관련 매파적 발언과 외국인 매수세 등이 강세 재료로 작용하여 국고채 금리는 강세를 보였다. 주 마지막 영업일 국내 채권시장은 전일 미 인플레이션 지표 둔화와 외국인 국 채선물 매수세로 인해 금리가 큰 폭으로 하락했고, 국고 3년 최종호가 수익률은 22년 하반기 이후 최저 점을 경신하는 등 강세를 보였다. 12일 국고채 3년 최종호가 수익률은 전장 대비 6.2bp 하락한 3.101%, 10년물은 5.7bp 하락한 3.144%를 나타냈다.', score=8.267670631408691, index=11),
 RerankResult(text='익영업일 또한 미 국채 강세와 더불어 매수세까지 더해지며 강세를 지속했고, 국고채 3년 최종호가 수익률은 2.900%를 하회하 는 등 하락세를 보였다. 주중 국고채 금리는 수급의 영향으로 꾸준히 하락했고, 다음 주 금통위에 대한 기대 심리 또한 강세 압력을 더했다. 주 마지막 영업일, 제롬 파월 의장의 매파적 발언이 미 국채 금리 상승을 이끌었고, 국내 채권시장 또한 소폭 약세를 보였다. 15일 국고채 3년 최종호가 수익률은 전장 대비 1.2bp 상승한 2.941%, 10년 최종호가 수익률은 1.0bp 상승한 3.084%로 마감했다. 국고 3년물은 지난주 대비 12.0bp 하락했다.', score=7.888547420501709, index=12),
 RerankResult(text="주 마지막 영업일 국내 채권시장은 국채선물 만 기와 긴 연휴를 앞두고 거래량이 미미한 모습을 보였다. 이날 연준의 비공식 대변인으로 불리는 월스트 리트저널의 닉 티미라오스 기자가 '빅컷' 인하 가능성을 우회적으로 제시했고, 관련 발언으로 시장에는 50bp 금리 인하 기대감이 재차 커지면서 강세 압력을 가했다. 13일 국고채 3년 최종호가 수익률은 전 장 대비 4.4bp 하락한 2

In [110]:
answer[15]

'12월 13일 국고채 3년 최종호가 수익률은 2.541%였습니다.'

In [111]:
from langchain_teddynote.evaluator import GroundednessChecker

question_answer_relevant = GroundednessChecker(
  llm=ChatOpenAI(model='gpt-4o-mini', temperature=0), target='question-retrieval'
).create()

# response_ = []
# for j in answer:
#     response = []
#     for i in b:
#         response.append(question_answer_relevant.invoke(
#         {'context': i, 'answer': answer[-1]}
#         ))
#     response_.append(response)

In [113]:
response_ = []
yes_thing = []
for j, k, l in zip(batch_dataset, context, invoked_list):
    answ = []
    for i in k:
        for o in l:
            if i.text == o.page_content:
                answ.append(o)
    a = set()
    for i in answ:
        a.add((i.metadata['page'], i.metadata['source']))

    b = []
    for i, m in a:
        for n in splitted_doc:
            if n.metadata['page'] == i and n.metadata['source'] == m:
                b.append(n)

    response = []
    for i in b:
        result =question_answer_relevant.invoke(
        {'context': i, 'question': j}
        )
        response.append(result)
        if result.score=='yes':
            yes_thing.append(i)
    response_.append(response)

In [114]:
# 6 6 14 16 16 16 17 17 17 17 17 17 17 20 20 20 20 20 23
response_

[[GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no')],
 [GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no')],
 [GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(score='no'),
  GroundnessQuestionRetrievalScore(sco

In [119]:
ttt = []
for i, j in enumerate(response_):
  for k in j:
    if k.score =='yes':
      ttt.append((i))

In [122]:
ttt

[6,
 6,
 14,
 14,
 14,
 14,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 20,
 20,
 20,
 20,
 20,
 23]

In [123]:
yes_thing[9:16]

[Document(metadata={'type': 'image', 'image': './raw_pdf_copy3\\KISWeekly제1113호(20241206)\\517.png', 'page': 34, 'source': './raw_pdf_copy3\\KISWeekly제1113호(20241206).pdf', 'id': 517, 'issue_date': '20241206'}, page_content='주간 ELS 발행현황에서 3년 만기 유형이 3,920억 원으로 가장 높은 발행금액을 기록했으며, 1년 만기는 1,757억 원, 0.5년 만기는 384억 원, 0.25년 만기는 396억 원으로 나타났고, 2년과 1.5년 만기는 각각 265억 원과 0으로 발행되지 않았음을 알 수 있습니다.'),
 Document(metadata={'type': 'image', 'image': './raw_pdf_copy3\\KISWeekly제1113호(20241206)\\198.png', 'page': 13, 'source': './raw_pdf_copy3\\KISWeekly제1113호(20241206).pdf', 'id': 198, 'issue_date': '20241231'}, page_content='2024년 12월 03일에 중금채가 약 15,000으로 가장 높았고, 산금채는 12,000으로 두 번째로 높았으며, AAA는 8,000, AA+는 5,000으로 나타났다; 2024년 12월 02일에는 중금채가 10,000, 산금채가 7,000으로 감소했으며, AAA는 6,000, AA+는 4,000이었다; 2024년 12월 04일에는 중금채가 12,000, 산금채가 10,000으로 증가했으며, AAA는 7,000, AA+는 5,000이었다; 2024년 12월 05일에는 중금채가 11,000, 산금채가 9,000으로 감소했고, AAA는 8,000, AA+는 6,000이었다; 2024년 12월 06일에는 중금채가 9,000, 산금채가 7,000으로 다시 감소했으며, AAA는 6,000,

In [129]:
invoked_list[17]

[Document(metadata={'source': './raw_pdf_copy3\\KISWeekly제1113호(20241206).pdf', 'summary': '- 2023년 12월 1일 기준 정기예금 발행금액: 12,153억 원 (전주 대비 748억 원 감소)\n- 365일물 기준 호가: 3.23%\n- 부동산 관련 대출 발행금액: 8,299억 원 (전주 대비 3,580억 원 감소)\n  - 수도권: 5,609억 원\n  - 기타 지역: 2,690억 원\n- PF물 90일 기준 발행금리: 3.80% (전주 대비 보합세)', 'issue_date': '20241206', 'pk': '992b515c-0921-41f1-950b-fb851b8460d2', 'type': 'text', 'page': 26}, page_content='금주 부동산 관련 대출 발행금액은 8,299 억원으로 전주 대비 3,580억원 감소했다. 그 중 수도권이 5,609 억 원, 그 외 지역이 2,690 억 원을 차지했다. PF물 90일 기준 발행금리는 3.80%로 전주 대비 보합세로 마무 리되었다.'),
 Document(metadata={'source': './raw_pdf_copy3\\KISWeekly제1113호(20241206).pdf', 'summary': '- 2023-12-01 기준, ABCP 및 ABSTB 유동화 발행금액: 130,998억원\n- 전주 대비 감소: 3,305억원\n- 기초자산: 부동산 관련 대출이 가장 많이 발행, 정기예금이 뒤를 이음\n- 자료 출처: KIS자산평가', 'issue_date': '20241206', 'pk': '8eead8ea-f75c-481e-b141-ca51bd15a6d3', 'type': 'text', 'page': 25}, page_content='금주 유동화 발행금액은 130,998억원으로 전주 대비 3,305억원 감소하였다. 기초자산은 부동산 관련 대 출이 가장 많이 발행되었고, 정기예금이 그 뒤를 이었다.'),
 Documen

In [125]:
answer[17]

'2023년 12월 첫째 주 대출 발행금액은 8,299억 원으로, 전주 대비 3,580억 원 감소했습니다.'

In [ ]:
ground_truth = df_qa["ground_truth"]
question = df_qa["question"]
answer = pd.Series(answer)

question = df_qa["question"]
combined_df = pd.concat([question,ground_truth, answer], axis=1)
combined_df.columns = ['question',"ground_truth", "answer"]
combined_df.to_excel("combined_df.xlsx", index=False)

In [ ]:
!pip install --upgrade pymilvus # 필요한 종속성 설치

  Using cached pymilvus-2.5.5-py3-none-any.whl.metadata (5.7 kB)
Using cached pymilvus-2.5.5-py3-none-any.whl (223 kB)
  Attempting uninstall: pymilvus
    Found existing installation: pymilvus 2.5.4
    Uninstalling pymilvus-2.5.4:
      Successfully uninstalled pymilvus-2.5.4


In [19]:
!pip install "pymilvus[model]"


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 43.2/43.2 kB 2.1 MB/s eta 0:00:00


In [20]:
!pip show milvus-model

Name: milvus-model
Version: 0.2.12
Summary: Model components for PyMilvus, the Python SDK for Milvus
Home-page: 
Author: 
Author-email: Milvus Team <milvus-team@zilliz.com>
License: 
Location: C:\Users\Jo\AppData\Local\Programs\Python311\Lib\site-packages
Requires: numpy, onnxruntime, protobuf, scipy, transformers
Required-by: 


In [ ]:
result_ = []
batch_dataset = [question for question in test_dataset["question"]]
context = []
answer = []

for expr, question in zip(expressions, batch_dataset):
    invoked_milvus = milvus_retriever.invoke(question, expr=expr)
    invoked_bm25 = bm25.invoke(question)
    invoked_filtered = []
    for i in invoked_bm25:
        if expr != None:
            if i.metadata['issue_date'] >= expr.split("'")[1] and i.metadata['issue_date'] <= expr.split("'")[3]:
                invoked_filtered.append(i)
        else:
            invoked_filtered.append(i)
    invoked_bm25_2 = bm25_2.invoke(question)
    invoked =  invoked_milvus + invoked_bm25_2 + invoked_filtered
    document_texts = [doc.page_content for doc in invoked]
    reranked_results = bge_rf(
        query=question,
        documents=document_texts,
        top_k=10
    )
    context.append(reranked_results)
    chain = (
        {'context' : itemgetter('context'), 'question': itemgetter('question')}
        # | add_date_to_inputs
        | prompt
        | llm
        | StrOutputParser()
    )
    answer.append(chain.invoke({'question':question, 'context': invoked}))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
if "contexts" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["contexts"]).add_column("contexts", [[j.text for j in i] for i in context])
else:
    test_dataset = test_dataset.add_column("contexts", [[j.text for j in i] for i in context])

if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)
else:
    test_dataset = test_dataset.add_column("answer", answer)

test_dataset = test_dataset.map(convert_to_list)

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    context_recall,
    context_precision,
    AnswerCorrectness,
    answer_similarity
)

for _ in range(3):
    result = evaluate(
        dataset=test_dataset,
        metrics=[
            context_recall,
            context_precision,
            answer_similarity,
            AnswerCorrectness(weights=[0.75, 0.25]),
        ],
    )
    result_.append(result)

with open('./score/250310_bge.json', "w", encoding='utf-8') as json_file:
    json.dump(result_, json_file, indent=4)

In [ ]:
ground_truth = df_qa["ground_truth"]
question = df_qa["question"]
answer = pd.Series(answer)

question = df_qa["question"]
combined_df_bge = pd.concat([question,ground_truth, answer], axis=1)
combined_df_bge.columns = ['question',"ground_truth", "answer"]
combined_df_bge.to_excel("combined_df_bge.xlsx", index=False)